In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Load data from the Excel file
df = pd.read_excel('/Users/marclambertes/Downloads/MFF BENE.xlsx')
#df = df[df['Minutes played'] >= 900]
df = df[df['Position'].notna()]  # Exclude rows with empty values in the "Position" column
df = df[df['Position'].str.contains('CF')]
# List of teams to exclude
teams_to_exclude = ['Ajax', 'PSV', 'Feyenoord', 'AZ', 'Twente', 
                    'Club Brugge', 'Gent', 'Genk', 'Anderlecht', 'Antwerp']

# Filter out rows where the 'Team' column is in the list of teams to exclude
df = df[~df['Team'].isin(teams_to_exclude)]

# Define the list of metrics for which you want to calculate percentile ranks
metrics = ['Goals per 90', 'Non-penalty goals per 90', 'xG per 90', 'Head goals per 90', 'Shots per 90', 'Shots on target, %',
           'Assists per 90', 'xA per 90', 'Crosses per 90', 'Dribbles per 90', 'Successful dribbles, %', 'Offensive duels won, %',
           'Touches in box per 90', 'Progressive runs per 90']
#metrics = ['Shots against per 90', 'Save rate, %', 'Saves with reflex, %', 
 #          'Prevented goals per 90', 'Aerial duels per 90', 'Exits per 90',
  #         'Passes per 90', 'Accurate passes, %']

# Calculate percentile ranks for each metric and add them as new columns
for metric in metrics:
    percentile_rank_column_name = f'{metric}'
    df[percentile_rank_column_name] = df[metric].rank(pct=True) * 100

# Select only the desired columns in the DataFrame
selected_columns = ['Player', 'Team', 'Age', 'Minutes played'] + [f'{metric}' for metric in metrics]

df = df[selected_columns]

# Create a new Excel workbook
wb = Workbook()

# Select the active worksheet (you can change this to any other sheet if needed)
ws = wb.active

# Add the DataFrame to the worksheet
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)

# Save the new Excel file with the selected columns and percentile ranks
output_excel_file = '/Users/marclambertes/Downloads/MFF PER.xlsx'
wb.save(output_excel_file)

print(f'Selected columns and percentile ranks have been saved to {output_excel_file}.')


Selected columns and percentile ranks have been saved to /Users/marclambertes/Downloads/MFF PER.xlsx.


In [3]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Load data from the Excel file
df = pd.read_excel('/Users/marclambertes/Downloads/MFF BENE.xlsx')
df = df[df['Position'].notna()]  # Exclude rows with empty values in the "Position" column
df = df[df['Position'].str.contains('CF')]
# List of teams to exclude
teams_to_exclude = ['Ajax', 'PSV', 'Feyenoord', 'AZ', 'Twente', 
                    'Club Brugge', 'Gent', 'Genk', 'Anderlecht', 'Antwerp']

# Filter out rows where the 'Team' column is in the list of teams to exclude
df = df[~df['Team'].isin(teams_to_exclude)]

# Define the list of metrics for which you want to calculate z-scores
metrics = ['Goals per 90', 'Non-penalty goals per 90', 'xG per 90', 'Head goals per 90', 'Shots per 90', 'Shots on target, %',
           'Assists per 90', 'xA per 90', 'Crosses per 90', 'Dribbles per 90', 'Successful dribbles, %', 'Offensive duels won, %',
           'Touches in box per 90', 'Progressive runs per 90']

# Calculate z-scores for each metric and add them as new columns
for metric in metrics:
    z_score_column_name = f'{metric} Z-Score'
    z_scores = (df[metric] - df[metric].mean()) / df[metric].std()
    # Transform z-scores to a 0-100 scale with 50 being the mean
    df[z_score_column_name] = 50 + 10 * z_scores

# Create a new column for the average z-score
df['Average Z-Score'] = df[[f'{metric} Z-Score' for metric in metrics]].mean(axis=1)

# Reorder columns with the new z-score and average z-score columns
selected_columns = ['Player', 'Team', 'Age', 'Minutes played'] + [f'{metric} Z-Score' for metric in metrics] + ['Average Z-Score']

df = df[selected_columns]

# Create a new Excel workbook
wb = Workbook()

# Select the active worksheet (you can change this to any other sheet if needed)
ws = wb.active

# Add the DataFrame to the worksheet
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)

# Save the new Excel file with the selected columns, transformed z-scores, and average z-score
output_excel_file = '/Users/marclambertes/Downloads/MFF Z.xlsx'
wb.save(output_excel_file)

print(f'Selected columns, transformed z-scores, and average z-score have been saved to {output_excel_file}.')


Selected columns, transformed z-scores, and average z-score have been saved to /Users/marclambertes/Downloads/MFF Z.xlsx.


In [13]:
import pandas as pd

# Load your Excel file into a DataFrame
df = pd.read_excel('striker_zscores_swiss_super_league.xlsx')

# Group by both 'Player' and 'Team within selected timeframe'
grouped = df.groupby(['Player', 'Team within selected timeframe'])

# Calculate the average for each column within each group
average_per_player = grouped.mean().reset_index()

# Now 'average_per_player' contains the average for each column for each player and team within the selected timeframe
print(average_per_player[['Player', 'Team within selected timeframe', 'Age', 'Average Z-Score']])


            Player Team within selected timeframe   Age  Average Z-Score
0        A. Gruber                   Austria Wien  27.0        52.673499
1       A. Lutovac                           Ried  25.0        45.689413
2         A. Nuhiu               Rheindorf Altach  33.0        47.626874
3        A. Schmid               Austria Lustenau  24.0        49.430586
4        A. Tibidi               Rheindorf Altach  19.0        51.472348
5      B. Teixeira                     Sturm Graz  22.0        53.141676
6    B. Zimmermann                     Rapid Wien  21.0        51.864619
7         B. Šeško                       Salzburg  20.0        53.932355
8          C. Lang                           Ried  21.0        47.425762
9     C. Monschein                           Ried  30.0        44.175978
10      D. Avdijaj                       Hartberg  26.0        53.996292
11         D. Fitz                   Austria Wien  24.0        49.776386
12        D. Tadić                       Hartberg  

In [15]:
import pandas as pd

# Load your Excel file into a DataFrame
df = pd.read_excel('striker_zscores_eredivisie.xlsx')

# Group by both 'Player' and 'Team within selected timeframe'
grouped = df.groupby(['Player', 'Team within selected timeframe'])

# Calculate the average for each column within each group
average_per_player = grouped.mean().reset_index()

# Sort the DataFrame by the 'Average Z-Score' column in descending order
sorted_df = average_per_player.sort_values(by='Average Z-Score', ascending=False)

# Select the top 15 rows
top_15 = sorted_df.head(15)

# Write the top 15 rows to an Excel file
top_15.to_excel('top_15_eredivisie.xlsx', index=False)
